In [19]:
! pip install langchain openai streamlit python-dotenv langchain_community -q

print('Installaion successful')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
Installaion successful


Loads any existing .env variables.

Compares them to your desired list (env_keys).

If any are missing but exist in the system environment, it appends them to the file.

In [16]:
! pwd
! ls
! ls -a

/content
sample_data
.  ..  .config	.env  sample_data


In [17]:
# env file handel
import os

env_file = ".env"
env_keys = ["API_KEY", "DATABASE_URL", "SECRET_KEY"]  # Customize this list

existing_vars = {}

# Load existing variables if file exists
if os.path.exists(env_file):
    with open(env_file, "r") as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith("#") and "=" in line:
                key, val = line.split("=", 1)
                existing_vars[key.strip()] = val.strip()
    print(f"Loaded existing variables from {env_file}.")
else:
    # Create the file if it doesn't exist
    with open(env_file, "w") as f:
        f.write("")  # Create an empty file
    print(f"Created new {env_file} file.")

# Collect missing vars
new_vars = {}
for key in env_keys:
    if key not in existing_vars:
        value = os.getenv(key)
        if not value:
            # Optional: prompt the user for a value if it's missing
            value = input(f"Enter value for {key}: ").strip()
        if value:
            new_vars[key] = value

# Write new vars to file
if new_vars:
    with open(env_file, "a") as f:
        for key, value in new_vars.items():
            f.write(f"{key}={value}\n")
    print(f"Added to {env_file}: {', '.join(new_vars.keys())}")
else:
    print("No new variables to add.")


Loaded existing variables from .env.
No new variables to add.


In [33]:
from langchain.llms import OpenAI
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

load_dotenv()

api_key = os.getenv("API_KEY")
api_base = os.getenv("DATABASE_URL")

model1="deepseek/deepseek-chat-v3-0324:free"
model2="google/gemini-2.5-pro-exp-03-25:free"
model3="deepseek/deepseek-r1:free"
model4="deepseek/deepseek-r1-zero:free"
def generate_pet_name():
  # llm = OpenAI(openai_api_key=api_key,openai_api_base=api_base,temperature=0.7)
  # prompt="I have a dog pet and I want a cool anme for it. Suggest me five cool names for my pet."
  # name= llm(prompt)
  # return name
  llm=ChatOpenAI( openai_api_key=api_key, openai_api_base=api_base,model_name=model2,temperature=0.7)
  prompt=messages = [HumanMessage(content="I have a dog pet and I want a cool anme for it. Suggest me five cool names for my pet.")]
  name=llm(prompt)
  return name


if __name__ =="__main__":
  print(generate_pet_name())

content="Okay, here are five cool name suggestions for your dog, hitting a few different styles:\n\n1.  **Maverick:** Sounds independent, adventurous, and a bit rebellious. Popularized by *Top Gun*, it has a strong, confident feel.\n2.  **Zephyr:** Unique and sounds sleek (it means a gentle breeze). It has a cool, almost ethereal vibe but is still easy to say.\n3.  **Atlas:** Strong, grounded, and carries a bit of weight (from the mythological Titan who held up the sky). It sounds significant and dependable.\n4.  **Onyx:** Named after the black gemstone, this sounds sleek, cool, and a little mysterious. Especially great for a dark-coated dog, but cool for any.\n5.  **Jax:** Short, punchy, and has a modern, slightly edgy feel. It's energetic and easy to call out.\n\n**Bonus Tip:** Try saying the names out loud as if you were calling your dog. See which one feels right and rolls off the tongue easily! Good luck choosing!" additional_kwargs={} response_metadata={'token_usage': {'completio

In [22]:
%%writefile gen.py
import os
from langchain.llms import OpenAI
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

load_dotenv()

api_key = os.getenv("API_KEY")
api_base = os.getenv("DATABASE_URL")

model1="deepseek/deepseek-chat-v3-0324:free"
model2="google/gemini-2.5-pro-exp-03-25:free"
model3="deepseek/deepseek-r1:free"
model4="deepseek/deepseek-r1-zero:free"

def generate_pet_name(animal_type, pet_color):
  # llm = OpenAI(openai_api_key=api_key,openai_api_base=api_base,temperature=0.7)
  # prompt="I have a dog pet and I want a cool anme for it. Suggest me five cool names for my pet."
  # name= llm(prompt)
  # return name
  llm=ChatOpenAI( openai_api_key=api_key, openai_api_base=api_base,model_name=model2,temperature=0.7)

  prompt_template_name=PromptTemplate(
      input_variables=['animal_type', 'pet_color'],
      template="I have a {animal_type} pet and I wnat a col name for it, the color of my pet is {pet_color}. suggest me five cool names for my pet."
  )

  name_chain = LLMChain(llm=llm,prompt=prompt_template_name,output_key="pet_name")
  response=name_chain({'animal_type':animal_type, 'pet_color':pet_color})
  return response


if __name__ =="__main__":
  print(generate_pet_name("cat","brown"))

Overwriting gen.py


tutorial for streamlit on colab ("https://youtu.be/NEhrkeF2o_M?si=5AHh3xxhqGIDWXZd")

In [23]:
%%writefile app.py
import streamlit as st
import gen as g
st.title("My pet name generator")

uanimal_type=st.sidebar.selectbox("What is your pet?",("cat","Dog","cow","Hamster"))
uanimal_color=st.sidebar.selectbox("what color is your "+uanimal_type,("black","brown","white","grey"))

if st.sidebar.button("Generate pet name"):
  response=g.generate_pet_name(uanimal_type,uanimal_color)
  st.write(response['pet_name'])




Overwriting app.py


In [24]:
!streamlit run app.py & npx localtunnel --port 8501 -y



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.194.137.139:8501

⠴⠦⠧⠇⠏⠋⠙⠹⠸your url is: https://fifty-camels-allow.loca.lt
/usr/local/lib/python3.11/dist-packages/langchain/llms/__init__.py:549: LangChainDeprecationWarning: Importing LLMs from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.llms import OpenAI`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/langchain/llms/__init__.py:549: LangChainDeprecationWarning: Importing LLMs from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.llms import OpenAI`.

To install langchain-community r